# 4D-STEM Tutorial

This tutorial is designed as a quick walk-through to the functions avaible through pyXEM.  It will not cover everything and if you want to further explore pyxem a better place to satart is with the pyxem-demos which will offer more indepth instrucution on using pyxem. 

This demo is designed to work with pyxem version 13.3.  If you don't have version 13.3 installed it is a good idea to update your version of pyxem so that things work as intended. 

-- csfrancis@wisc.edu --

Monday July 26, 2021


## Contents 

0. <a href='#in'> Introduction</a>
1. <a href='#pr'> Loading pyXEM</a>
2. <a href='#vdf'> Virtual Dark Feild Imaging</a>
3. <a href='#vf'> Peak Finding and Machiene Learning</a>
4. <a href='#ml'> Orentation Mapping</a>
5. <a href='#vec'> Amorphous Materials</a>

<a href='#in'></a>

## Introduction:

### What is pyXem?

- Pyxem is part of the `Hyperspy` ecosystem specifically designed for analyzing diffraction from pixelated detectors.

### What about 4-D STEM?

- Yep! 4-D STEM, 5-D STEM and on.  Pyxem is purposely built to handle any size data that you have. 

### What is avabile in pyXEM?

- Orientation Mapping of Crystals
- Strain Mapping
- Virtual Dark Field Imaging
- Differential Phase Contrast (DPC)
- Structural Characterization of Amorphous Materials (Angular Correlations/FEM)
- And much more!

###  Is pyXEM  still in Development?

- PyXEM is still in a beta format which means that with every minor relase there will still be API changes.  We try to keep these to a minimum but they will still occur.

- A major relase is upcoming which should stablize pyXEM's API in the near future. 

### Is pyXEM part of Hyperspy?

- Yes! and No... 
- pyXEM **extends** the capabilities of Hyperspy.  With the new Hyperspy-bundle pyXEM is automatically included which means you won't have to deal with downloading pyXEM seperately!

<br/>
<br/>
<img style='margin:auto;' src='img/HyperspyExtension.png'/>
<br/>

In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import diffpy

<a id='pr'></a>

## Printing Known Signal Types

Pyxem **extends** hyperspy.  This means that for most operations you won't have to load pyxem once pyxem is downloaded.  With the [``hyperspy bundle``](http://hyperspy.org/hyperspy-doc/current/user_guide/install.html#hyperspy-bundle) this is very easy as you can use anaconda to easily download and install the necessary packages. 

A good first step in any operation is to print the known signal types. This is very helpful to:

**1- Check that packages like pyxem/kikuchipy/atomap etc. are loaded**

**2- Use as a reference when changing signal types**

In [ ]:
# Printing the known signal types

<a href='#lo'></a>
## 1. Loading & Visualization

Loading data into pyxem is just as easy as loading data into hyperspy. We can call the load function from hyperspy and load the data. 

See how the data is loaded as a Diffraction2D Signal already, this is because hyperspy remembers the Signal type if you save the data. 

In [ ]:
# Loading data 
dp = hs.load('./data/twinned_nanowire.hdf5')
#print the signal

### 1.1 Changing Signal Type
You can also change the Signal type after the fact.  In this case we want to work with electron diffraction data so we use the `set_signal_type` function to change the signal using the alais shown above from the `print_known_signals` function.

In [ ]:
# set the signal type to 'electron_diffraction'

### 1.2 Adjusting the Axes Scale & Metadata

In many cases data will not be loaded with appropiate units/ experimental parameters or scale.  Hyperspy is usually pretty good at reading this from the file but often times that data will have to be added after. We can fix this by manually setting this for our dataset.

In [ ]:
dp.beam_energy = 300
dp.metadata.Acquisition_instrument.TEM.camera_length =21 
dp.metadata.Acquisition_instrument.TEM.scan_rotation = 277
dp.metadata.Acquisition_instrument.TEM.convergence_angle =0.7
dp.metadata.Acquisition_instrument.TEM.exposure_time =10.0
dp.metadata.General.title = 'GaAs Nanowire'

In [ ]:
# Show the metadata that we are interested in

### 1.3 Applying an Affine Transformation and Centering the Direct Beam 

In most data some of your first steps are going to be applying an affine correction (to correct for astigmatism if present) and then centering the direct beam. (This can also help to correct for any scan pivot point misalignments which are often difficult to correct for.)

If you want to determine the proper affine correction using a standard is very helpful.  Refer to the `pyXEM` [User Guide](https://github.com/pyxem/pyxem-demos) for help. 

**Note:** This won't work if you have a beam stop blocking the zerobeam.


In [ ]:
#look at the center direct beam function

In [ ]:
transform = np.array([[0.99,0,0], [0,0.69,0],[0,0,1]])

In [ ]:
# plot one pattern from original data


In [ ]:
#apply an affine transformation
dp.apply_affine_transformation(transform,
                               keep_dtype=True,
                               inplace=True)

# Centering the Direct Beam
dp.center_direct_beam(method='cross_correlate',
                                             radius_start=2,
                                             radius_finish=5,
                                             half_square_width=10,
                                 )
#reset the offset for the axes
dp.axes_manager[2].offset = dp.axes_manager[2].size/2
dp.axes_manager[3].offset = dp.axes_manager[3].size/2

In [ ]:
# plot onr pattern from transformed data
dp.inav[10,10].plot(vmax=50)
plt.show()

### 1.3 Adjusting the Axes Scale & Metadata

In many cases data will not be loaded with appropiate units/ experimental parameters or scale.  Hyperspy is usually pretty good at reading this from the file but often times that data will have to be added after. We can fix this by manually setting this for our dataset.

In this case we want to look at the inter planer spacing to determine the scale bar.

#### Side Note:

If you want to do this interactively you need to make sure you are using a backend for matplotlib which is interactive.  For the most part ``%matplotlib notebook`` should work.

Interact with the plot as necessary and then if you compute the cell below it will update with line profile of the area of interest. 

In [ ]:
%matplotlib notebook

In [ ]:
#take the mean of the diffraction pattern

In [ ]:
# Creating a Line ROI to get spacing 

In [ ]:
trace = line(dpm)
trace = trace.as_signal1D(spectral_axis=0)
trace.plot()

In [ ]:
#find the peaks in the trace.


In [ ]:
#plot the peaks.
trace.plot(plot_markers=True)

In [ ]:
#plot the peaks.

markers = [hs.plot.markers.vertical_line(x=peak[0], color='red') for peak in peaks[0]]
tmarkers = [hs.plot.markers.text(x=peak[0], y=50,text=np.round(peak[0],3), color='red') for peak in peaks[0]]
trace.add_marker(markers, permanent=True)
trace.add_marker(tmarkers, permanent=True)

In [ ]:
# setting the calibration
pixel_d111 = (67.242-24.542)/4
GaAs_dspacing =5.75018263
recip_d111 = np.sqrt((3/GaAs_dspacing**2))
recip_cal = recip_d111 / pixel_d111
dp.set_scan_calibration(10)
dp.set_diffraction_calibration(recip_cal)
dp.unit = "k_A^-1"

In [ ]:
#visualize the axes manager to make sure that the function works...

<a href='#vdf'></a>

## 2. Virtual Dark Field (VDF) Imaging
Now it is pretty easy to create a Virtual Diffraction Image using hyperspy's ROI tools.  In this case we just use the `plot_integrated_intensity` function to view our roi.

In [ ]:
#Creating an Interactive VDF Imange


In [ ]:
vdf = dp.get_integrated_intensity(roi)

In [ ]:
vdf.plot()
print(vdf)

## 2.1 Forming Multiple Images from Concentric Rings!

From pyxem we can also load the virtual Image generator to form images from different annular ranges. (In many different ways!)

### Using the Azimuthal Integrator 

One of my favorite ways to look at data is in polar coordinates.  This allows us to better see things like k-spacing and crystal orientations.

Pyxem uses [PyFAI](https://pyfai.readthedocs.io/en/master/) to handle all of the backend azimuthal integration.  This means that pyxem can handle:

- Unique detector configerations
- GPU acclerated integration
- Many different algorithms implemented by PyFAI. 

In [ ]:
# Setting the Azimuthal Integrator to help unwrap the function

In [ ]:
# get the Azimuthal Integral1d

In [ ]:
#get the 2D integral and then sum along the k direction
radial_rep = dp.get_azimuthal_integral2d(npt=50, npt_azim=90).sum(axis=3).T
radial_rep.axes_manager[0].units= "Rad"
radial_rep.plot()

## 3.0 Using Machiene Learning:

We are going to use Non-negitivie Matrix factorization (NMF) to help segement our image. 

This is particularly useful becuase unlike SVD or PCA it applies the constraint that every value needs to be positive (which is particualarly important in our case where we can assume that each component is positive. 

More information on what ```hyperspy``` is calling :

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html#sklearn.decomposition.NMF

In [ ]:
# preform NMF decomposition (try output_dimension =4)
# Try other dimensions if you are interested. How does it change?

In [ ]:
#plot decomposition loadinds and factors.  What does each loading and factor represent?

## 4.0 Peak Finding:

Pyxem also implements the hyperspy peak-finding algorithm which can be tuned to help identify diffraction spots.  

This helps define diffraction vectors which can be used for a variety of different things

In the example below we use the Difference of Gaussians (DOG) method from skimage to find the diffraction spots. 

https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_blob.html?highlight=dog


In [ ]:
#try adjusting the threshold here to find more or less peaks. 
# (Hint anything lower the 0.02 will fail take a long time to find all of the peaks)
peaks = dp.find_peaks(method='difference_of_gaussian',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.02,
                       overlap=0.99,
                       interactive=False)

In [ ]:
from pyxem.signals.diffraction_vectors import DiffractionVectors

In [ ]:
peaks = DiffractionVectors.from_peaks(peaks,center=(72,72),calibration=recip_cal)

In [ ]:
peaks.plot_diffraction_vectors_on_signal(dp, vmax=50)

In [ ]:
# get the number of diffraction vectors at each point (x,y)
crystim = peaks.get_diffracting_pixels_map(binary=False)
crystim.plot(cmap='viridis')

## 5.0 Phase and Orientation Mapping (Pattern Matching)

We will move on to looking at a polymorphic GaAs Nanowire with two different phases present.

In [ ]:
# Loading data and setting everything up
dp = hs.load('./data/polymorphic_nanowire.hdf5')
accelarating_voltage = 200  # kV
camera_length = 0.2  # m
diffraction_calibration = 0.032  # px / Angstrom
dp.data = dp.data.astype('float32')
dp.data = dp.data *(1 / dp.data.max())
scale_x = 0.995
scale_y = 1.031
offset_x = 0.631
offset_y = -0.351
dp.apply_affine_transformation(np.array([[scale_x, 0, offset_x],
                                         [0, scale_y, offset_y],
                                         [0, 0, 1]]))
dp.plot()

## 5.1 Subtracting the Background
Perform difference of gaussian background subtraction with various parameters on one selected diffraction pattern and plot to identify good parameters. 


In this method we are basically trying to use a spatial band pass filter to remove anything that isn't a diffraction spot.  Our parameters should have a minimum sigma smaller than the smallest diffraction spot and larger than the largest diffraction spot. 

In [ ]:
from pyxem.utils.expt_utils import investigate_dog_background_removal_interactive

In [ ]:
investigate_dog_background_removal_interactive?

In [ ]:
# set up a test area
dp_test_area = dp.inav[0, 0]
gauss_stddev_maxs = np.arange(2, 12, 0.2) # min, max, step
gauss_stddev_mins = np.arange(1, 4, 0.2) # min, max, step
investigate_dog_background_removal_interactive(dp_test_area,
                                               gauss_stddev_maxs,
                                               gauss_stddev_mins)

Remove background using difference of gaussians method with parameters identified above

In [ ]:
# subtract the background using Difference of Gaussians

In [ ]:
f= plt.figure(figsize=(9,5))
hs.plot.plot_images([dp_test_area,dp.inav[0,0]], fig=f, label=["Before Subtraction",
                                                               "After Subtraction"])

In [ ]:
dp.set_signal_type("electron_diffraction") #this is needed because of a bug in the code
dp.set_diffraction_calibration(diffraction_calibration)
dp.set_scan_calibration(10)

##  5.2 Pattern Matching
Pattern matching generates a database of simulated diffraction patterns and then compares all simulated patterns against each experimental pattern to find the best match

Import generators required for simulation and indexation

In [ ]:
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

from diffsims.generators.zap_map_generator import get_rotation_from_z_to_direction
from diffsims.generators.rotation_list_generators import get_grid_around_beam_direction

from pyxem.generators.indexation_generator import TemplateIndexationGenerator

### Define Library of Structures & Orientations

Define the crystal phases to be included in the simulated library

In [ ]:
structure_zb = diffpy.structure.loadStructure('./data/GaAs_mp-2534_conventional_standard.cif')
structure_wz = diffpy.structure.loadStructure('./data/GaAs_mp-8883_conventional_standard.cif')

In [ ]:
#creating a basic roations list
za110c = get_rotation_from_z_to_direction(structure_zb, [1,1,0])
rot_list_cubic = get_grid_around_beam_direction(beam_rotation=za110c, resolution=1, angular_range=(0,180))
za110h = get_rotation_from_z_to_direction(structure_wz, [1,1,0])
rot_list_hex = get_grid_around_beam_direction(beam_rotation=za110h, resolution=1, angular_range=(0,180))

In [ ]:
# Creating a Strucutre Library Object defininf the crystal strcutures which will be simulated. 
struc_lib = StructureLibrary(['ZB','WZ'],
                             [structure_zb,structure_wz],
                             [rot_list_cubic,rot_list_hex])

In [ ]:
diff_gen = DiffractionGenerator(accelerating_voltage=accelarating_voltage)
lib_gen = DiffractionLibraryGenerator(diff_gen)

In [ ]:
target_pattern_dimension_pixels = dp.axes_manager.signal_shape[0]
half_size = target_pattern_dimension_pixels // 2
reciprocal_radius = diffraction_calibration*(half_size - 1)
diff_lib = lib_gen.get_diffraction_library(struc_lib,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=(half_size, half_size),
                                           max_excitation_error=1/10,
                                           with_direct_beam=False)

## 5.3. Pattern Matching Indexation
Initialize `TemplateIndexationGenerator` with the experimental data and diffraction library and perform correlation, returning the `n_largest` matches with highest correlation.

<div class="alert alert-block alert-warning"><b>Note:</b> This workflow has been changed from previous version, make sure you have pyxem 0.13.0 or later installed</div>

In [ ]:
indexer = TemplateIndexationGenerator(dp, diff_lib)
indexation_results = indexer.correlate(n_largest=3)

crystal_map is now a CrystalMap object, which comes from orix, see their documentation for details. Below we lift their code to plot a phase map

In [ ]:
crystal_map = indexation_results.to_crystal_map()

In [ ]:
from matplotlib import pyplot as plt
from orix import plot

fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
im = ax.plot_map(crystal_map)
print(crystal_map)

<a id='tem'></a>
##  6.0 Angular Correlations and FEM

###  6.1 Data Loading and Visualization

Below we are going to load a dataset for a PdNiP glass and look at the dominate symmetries in the glass.

To do this we cacluate the angular correlation defined below. 

<p style="text-align: center;">
$ C(k,\phi) = \frac{<I(k, \theta)*I(k, \theta+\phi)>_\theta - <I(k,\theta)>^2_\theta }{<I(k, \theta)>^2_\theta} $
</p>

This gives us the symmetry of some diffraction pattern regaurdless of orientation. 

In [ ]:
dp = hs.load("./data/PdNiP_test.hspy")

In [ ]:
dp.set_signal_type("electron_diffraction")
dp.beam_energy=200
dp.unit = "k_nm^-1"
# Affine correction from fitting an ellipse
import numpy as np
center=(31.2,31.7)
affine=np.array([[ 1.03725511, -0.02662789,  0.        ],
                 [-0.02662789,  1.01903215,  0.        ],
                 [ 0.        ,  0.        ,  1.        ]])

In [ ]:
dp.plot(vmax=5000)

In [ ]:
#set ai 
dp.set_ai(center=center)


In [ ]:
#get the azimuthal integral


In [ ]:
dp.sum().plot()

In [ ]:
#create a mask
summed = dp.sum()
mask = ((summed.data>4e6) +(summed.data<3e5))

In [ ]:
#plot the mask


In [ ]:
# plot the polar image

In [ ]:
#get the angular correlation

In [ ]:
#plot the angular correlation

In [ ]:
# get the angular power spectrum


In [ ]:
#plot the angular symmetries. 
import matplotlib.pyplot as plt
f = plt.figure(figsize=(15,10))
dp.plot_symmetries(k_region = [3.,4.5],fig=f)